# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

gmaps.configure(api_key = gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_sum_path = "../output_data/cities.csv"
city_sum = pd.read_csv(city_sum_path)
city_sum

,City Name,Longitude,Latitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,Date Time
0,Novosheshminsk,51.23,55.07,65.34,94,99,6.80,RU,1594413928
1,Villarrica,-56.43,-25.75,65.71,84,0,6.15,PY,1594413928
2,Ushuaia,-68.30,-54.80,33.80,74,40,5.82,AR,1594413890
3,Ponta do Sol,-17.10,32.67,72.00,72,3,6.51,PT,1594413929
4,Punta Arenas,-70.92,-53.15,33.80,86,75,9.17,CL,1594413843
...,...,...,...,...,...,...,...,...,...
538,Mehrān,46.16,33.12,96.93,11,0,10.00,IR,1594414049
539,Carballo,-8.69,43.21,64.40,72,40,8.05,ES,1594413793
540,Kindu,25.95,-2.95,78.19,51,31,0.98,CD,1594414049
541,Ilulissat,-51.10,69.22,42.80,87,100,12.75,GL,1594414019


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_sum[["Latitude", "Longitude"]]
humid = city_sum["Humidity"]

In [16]:
fig = gmaps.figure()
humidity_heat_map = gmaps.heatmap_layer(locations, weights=humid,
                                        dissipating=False, max_intensity=10,
                                        point_radius=3)
fig.add_layer(humidity_heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* A max temperature lower than 80 degrees but higher than 70.


* Wind speed less than 10 mph.


* Zero cloudiness.


* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [30]:
ideal_places = city_sum.loc[(city_sum['Max Temperature'] > 70) &
                            (city_sum['Max Temperature'] < 80) &
                            (city_sum['Wind Speed'] < 10) & 
                            (city_sum['Cloud Coverage'] == 0)]
ideal_places

,City Name,Longitude,Latitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,Date Time
122,Marsá Maţrūḩ,27.25,31.35,75.29,80,0,6.78,EG,1594413770
233,Kushmurun,64.63,52.45,74.68,77,0,6.38,KZ,1594413980
271,Portland,-122.68,45.52,79.00,60,0,1.01,US,1594413904
274,Florence,11.25,43.77,78.80,44,0,4.70,IT,1594413989
292,Esil,66.40,51.96,70.50,86,0,5.77,KZ,1594413993
301,Pagudpud,120.79,18.56,79.84,79,0,2.89,PH,1594413994
318,Corinth,22.96,37.94,73.99,42,0,3.00,GR,1594413998
351,Abu Samrah,37.18,35.30,73.42,72,0,9.93,SY,1594413926
355,Tovste,25.73,48.85,71.01,72,0,1.99,UA,1594414006
428,Mayskiy,40.10,47.69,71.60,43,0,8.95,RU,1594414022


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": locations,
    "rankby": "distance",
    "type": "hotel",
    "key": gkey}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
